In [1]:
import requests
import polars as pl

In [22]:
# [(dose4, Seed)]
# herbs = [
#     ("Toadflax", "Toadflax seed"), ("Harralander", "Harralander seed"), ("Tarromin", "Tarromin seed"),
#     ("Marrentill", "Marrentill seed"), ("Ranarr weed", "Ranarr seed"),
#     ("Irit leaf", "Irit seed"), ("Avantoe", "Avantoe seed"), ("Kwuarm", "Kwuarm seed")
# ]

# https://oldschool.runescape.wiki/w/Item_IDs
details_url = "https://secure.runescape.com/m=itemdb_oldschool/api/catalogue/detail.json"

useful_keys = ["current"]
herbs = [(3034, 3032), (2454, 2452), (121, 2428),
         (22464, 22461), (22452, 22449), (9741, 9739), (6472, 6470),
         (133, 2432), (24638, 24635), (24626, 24623), (23748, 23745),
         (23736, 23733), (23700, 23697), (23688, 23685), (23724, 23721),
         (23712, 23709)]
results = {}

for dose3, dose4 in herbs:
    dose3_data, dose4_data = None, None
    # dose3
    dose3_response = requests.get(details_url, params={"item":dose3})
    if dose3_response.status_code == 200:
        dose3_data = dose3_response.json()["item"]
        print(dose3_data["name"])
    # dose4
    dose4_response = requests.get(details_url, params={"item":dose4})
    if dose4_response.status_code == 200:
        dose4_data = dose4_response.json()["item"]
    if not dose3_data or not dose4_data:
        print(f"Error with {dose3}:\n\t{dose3_response.status_code}\n\t{dose4_response.status_code}")
    else:
        results[dose3_data["name"]] = (
            [dose3_data[key] for key in dose3_data.keys() if key in useful_keys],
            [dose4_data[key] for key in dose4_data.keys() if key in useful_keys]
        )
    # break

Agility potion(3)
Antifire potion(3)
Attack potion(3)
Bastion potion(3)
Battlemage potion(3)
Combat potion(3)
Compost potion(3)
Defence potion(3)
Divine bastion potion(3)
Divine battlemage potion(3)
Divine magic potion(3)
Divine ranging potion(3)
Divine super attack potion(3)
Divine super combat potion(3)
Divine super defence potion(3)
Divine super strength potion(3)


In [23]:
results["Agility potion(3)"]

([{'trend': 'neutral', 'price': 224}], [{'trend': 'neutral', 'price': 367}])

In [26]:
# Prepare data for DataFrame with correct handling of 'today' field

def parse_int(val):
    if isinstance(val, str):
        val = val.replace(',', '').strip().lower()
        val = val.replace(' ', '').replace('+', '')
        if val.endswith('k'):
            return int(float(val[:-1]) * 1_000)
        elif val.endswith('m'):
            return int(float(val[:-1]) * 1_000_000)
        try:
            return int(val)
        except ValueError:
            return None
    elif isinstance(val, (int, float)):
        return int(val)
    return None

data = []
for potion, (dose3_stats, dose4_stats) in results.items():
    row = {
        "potion": potion,
        "dose3_current": parse_int(dose3_stats[0]["price"]) if isinstance(dose3_stats[0], dict) and "price" in dose3_stats[0] else None,
        "dose4_current": parse_int(dose4_stats[0]["price"]) if isinstance(dose4_stats[0], dict) and "price" in dose4_stats[0] else None,
        "dose3_trend": dose3_stats[0]["trend"] if isinstance(dose3_stats[0], dict) and "trend" in dose3_stats[0] else None,
        "dose4_trend": dose4_stats[0]["trend"] if isinstance(dose4_stats[0], dict) and "trend" in dose4_stats[0] else None,
        # "dose3_today": parse_int(dose3_stats[1]["price"]),  # 'today' is a direct value, not a dict
        # "dose4_today": parse_int(dose4_stats[1]["price"]),  # 'today' is a direct value, not a dict
        # "dose3_day30": dose3_stats[2]["trend"] if isinstance(dose3_stats[2], dict) and "trend" in dose3_stats[2] else dose3_stats[2],
        # "dose4_day30": dose4_stats[2]["trend"] if isinstance(dose4_stats[2], dict) and "trend" in dose4_stats[2] else dose4_stats[2],
        # "dose3_day90": dose3_stats[3]["change"] if isinstance(dose3_stats[3], dict) and "change" in dose3_stats[3] else dose3_stats[3],
        # "dose4_day90": dose4_stats[3]["change"] if isinstance(dose4_stats[3], dict) and "change" in dose4_stats[3] else dose4_stats[3],
        # "dose3_day180": dose3_stats[4]["change"] if isinstance(dose3_stats[4], dict) and "change" in dose3_stats[4] else dose3_stats[4],
        # "dose4_day180": dose4_stats[4]["change"] if isinstance(dose4_stats[4], dict) and "change" in dose4_stats[4] else dose4_stats[4],
    }
    data.append(row)

df = pl.DataFrame(data)
df

potion,dose3_current,dose4_current,dose3_trend,dose4_trend
str,i64,i64,str,str
"""Agility potion(3)""",224,367,"""neutral""","""neutral"""
"""Antifire potion(3)""",1384,1980,"""neutral""","""neutral"""
"""Attack potion(3)""",8,21,"""neutral""","""neutral"""
"""Bastion potion(3)""",15800,21000,"""neutral""","""neutral"""
"""Battlemage potion(3)""",14200,19800,"""neutral""","""neutral"""
…,…,…,…,…
"""Divine ranging potion(3)""",6691,9294,"""neutral""","""neutral"""
"""Divine super attack potion(3)""",2293,5936,"""neutral""","""neutral"""
"""Divine super combat potion(3)""",19400,24700,"""neutral""","""neutral"""


In [27]:
# Calculate new columns as described

df = df.with_columns([
    (df["dose4_current"] / 4).alias("dose4_per_dose_current"),
    (df["dose3_current"] / 3).alias("dose3_per_dose_current"),
])
df

potion,dose3_current,dose4_current,dose3_trend,dose4_trend,dose4_per_dose_current,dose3_per_dose_current
str,i64,i64,str,str,f64,f64
"""Agility potion(3)""",224,367,"""neutral""","""neutral""",91.75,74.666667
"""Antifire potion(3)""",1384,1980,"""neutral""","""neutral""",495.0,461.333333
"""Attack potion(3)""",8,21,"""neutral""","""neutral""",5.25,2.666667
"""Bastion potion(3)""",15800,21000,"""neutral""","""neutral""",5250.0,5266.666667
"""Battlemage potion(3)""",14200,19800,"""neutral""","""neutral""",4950.0,4733.333333
…,…,…,…,…,…,…
"""Divine ranging potion(3)""",6691,9294,"""neutral""","""neutral""",2323.5,2230.333333
"""Divine super attack potion(3)""",2293,5936,"""neutral""","""neutral""",1484.0,764.333333
"""Divine super combat potion(3)""",19400,24700,"""neutral""","""neutral""",6175.0,6466.666667


In [13]:
# Select and sort the relevant columns
df.select([
    "potion",
    "dose4_per_dose_current",
    "dose3_per_dose_current"
]).sort([
    "dose4_per_dose_current",
    "dose3_per_dose_current"
], descending=True)


potion,dose4_per_dose_current,dose3_per_dose_current
str,f64,f64
"""Divine bastion potion(3)""",6600.0,6866.666667
"""Divine super combat potion(3)""",6175.0,6466.666667
"""Divine battlemage potion(3)""",6100.0,5600.0
"""Bastion potion(3)""",5250.0,5266.666667
"""Battlemage potion(3)""",4950.0,4733.333333
…,…,…
"""Defence potion(3)""",170.75,143.666667
"""Agility potion(3)""",91.75,74.666667
"""Combat potion(3)""",68.0,42.666667


In [29]:
# Display the difference between leaf8.56_minus_seed_today and the next row's leaf8.56_minus_seed
df = df.with_columns([
    (df["dose4_per_dose_current"] - df["dose3_per_dose_current"]).alias("per_dose_difference"),
])
df.sort([
    "per_dose_difference"
], descending=True)

potion,dose3_current,dose4_current,dose3_trend,dose4_trend,dose4_per_dose_current,dose3_per_dose_current,per_dose_difference
str,i64,i64,str,str,f64,f64,f64
"""Divine super defence potion(3)""",2352,6252,"""neutral""","""neutral""",1563.0,784.0,779.0
"""Divine super attack potion(3)""",2293,5936,"""neutral""","""neutral""",1484.0,764.333333,719.666667
"""Divine battlemage potion(3)""",16800,24400,"""neutral""","""neutral""",6100.0,5600.0,500.0
"""Divine super strength potion(3…",5673,9081,"""neutral""","""neutral""",2270.25,1891.0,379.25
"""Battlemage potion(3)""",14200,19800,"""neutral""","""neutral""",4950.0,4733.333333,216.666667
…,…,…,…,…,…,…,…
"""Attack potion(3)""",8,21,"""neutral""","""neutral""",5.25,2.666667,2.583333
"""Compost potion(3)""",1,3,"""neutral""","""neutral""",0.75,0.333333,0.416667
"""Bastion potion(3)""",15800,21000,"""neutral""","""neutral""",5250.0,5266.666667,-16.666667
